In [1]:
import os
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig, PeftModel, PeftConfig
from trl import SFTTrainer
import time
from dotenv import load_dotenv
import random
random.seed(42)

load_dotenv()

/Users/leo/.pyenv/versions/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

# Preparing Data for fine-tune

<br>
<br>
Using huggingface dataset "Text-to-sql-v1", we have the instruction, input and response. Next step is to create a prompt dataset to fine-tune llama2 open source model.
<br>
<br>

In [2]:
dataset = load_dataset("Clinton/Text-to-sql-v1")
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'response', 'source', 'text'],
        num_rows: 262208
    })
})

In [3]:
dataset_total = dataset['train']
len(dataset_total)

262208

In [4]:
dataset_total[0]

{'instruction': 'Name the home team for carlton away team',
 'input': 'CREATE TABLE table_name_77 (\n    home_team VARCHAR,\n    away_team VARCHAR\n)',
 'response': 'SELECT home_team FROM table_name_77 WHERE away_team = "carlton"',
 'source': 'sql_create_context',
 'text': 'Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables. ### Instruction: Name the home team for carlton away team ### Input: CREATE TABLE table_name_77 (\n    home_team VARCHAR,\n    away_team VARCHAR\n) ### Response: SELECT home_team FROM table_name_77 WHERE away_team = "carlton"'}

In [5]:
input_l = dataset_total[0]['text'].split("###")
input_l

['Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables. ',
 ' Instruction: Name the home team for carlton away team ',
 ' Input: CREATE TABLE table_name_77 (\n    home_team VARCHAR,\n    away_team VARCHAR\n) ',
 ' Response: SELECT home_team FROM table_name_77 WHERE away_team = "carlton"']

<br>
<br>
Follow the llama2 prompt template to include system instruction, the input query and output response in the training data. Next step is to do train, validation and test data split. However, since there are over 250,000 data points, due to computational limit, we just randomly pick 20,000 data points for fine-tuning.
<br>
<br>

In [6]:
df_list = []
system_prompt = """Below are sql tables schemas paired with instruction that describes a task. 
                Using valid SQLite, write a response that appropriately completes the request 
                for the provided tables. """
for i in range(len(dataset_total)):
    query = dataset_total[i]['instruction'] + dataset_total[i]['input']
    prompt = "[INST]<<SYS>>\n{}<</SYS>>\n\n{}[/INST]{}</s>".format(system_prompt, query, dataset_total[i]['response'])
    df_list.append(prompt)

In [7]:
sampled_df_list = random.sample(df_list, 20000)
sampled_df_list[0:1]

["[INST]<<SYS>>\nBelow are sql tables schemas paired with instruction that describes a task. \n                Using valid SQLite, write a response that appropriately completes the request \n                for the provided tables. <</SYS>>\n\nMy median and average answer score at given date. Reports median and average answer score. The MedianAnswerScore2 is more precise version of median.CREATE TABLE CloseReasonTypes (\n    Id number,\n    Name text,\n    Description text\n)\n\nCREATE TABLE ReviewTaskResultTypes (\n    Id number,\n    Name text,\n    Description text\n)\n\nCREATE TABLE PostFeedback (\n    Id number,\n    PostId number,\n    IsAnonymous boolean,\n    VoteTypeId number,\n    CreationDate time\n)\n\nCREATE TABLE ReviewTaskResults (\n    Id number,\n    ReviewTaskId number,\n    ReviewTaskResultTypeId number,\n    CreationDate time,\n    RejectionReasonId number,\n    Comment text\n)\n\nCREATE TABLE FlagTypes (\n    Id number,\n    Name text,\n    Description text\n)\n\nC

<br>
<br>
Doing a train, validation and test split with ratio 6:2:2
<br>
<br>

In [8]:
# Shuffle the list
random.shuffle(sampled_df_list)

# Calculate split indices
train_ratio = 0.6
val_ratio = 0.2

train_idx = int(len(sampled_df_list) * train_ratio)
val_idx = train_idx + int(len(sampled_df_list) * val_ratio)

# Split the data, transform it into huggingface dataset and store it locally
train = sampled_df_list[:train_idx]
val = sampled_df_list[train_idx:val_idx]
test = sampled_df_list[val_idx:]
train_dataset = Dataset.from_pandas(pd.DataFrame(train, columns=["text"]))
val_dataset = Dataset.from_pandas(pd.DataFrame(val, columns=["text"]))
test_dataset = Dataset.from_pandas(pd.DataFrame(test, columns=["text"]))
train_dataset.save_to_disk("train.hf")
val_dataset.save_to_disk("val.hf")
test_dataset.save_to_disk("test.hf")

Saving the dataset (1/1 shards): 100%|█| 12000/12000 [00:00<00:00, 2581242.53 ex
Saving the dataset (1/1 shards): 100%|█| 4000/4000 [00:00<00:00, 1542590.66 exam
Saving the dataset (1/1 shards): 100%|█| 4000/4000 [00:00<00:00, 1039287.37 exam


# Fine-tune (include Quantization)

<br>
<br>
Since I work on a mac and the mac chip currently does not support Quantization, I will exclude that part. If u need Quantization, uncomment the part below and uncomment the quantization_config in model loading the next section.
<br>
<br>

In [9]:
# compute_dtype = getattr(torch, "float16")

# quant_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=compute_dtype,
#     bnb_4bit_use_double_quant=False,
# )

base_model = "meta-llama/Llama-2-7b-chat-hf"
new_model = "Llama-2-7b-chat-hf-text2sql"
token = os.getenv('token')

<br>
<br>
Load model from huggingface
<br>
<br>

In [10]:
tokenizer = AutoTokenizer.from_pretrained(base_model, token = token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
model = AutoModelForCausalLM.from_pretrained(base_model, 
                                            #quantization_config=quant_config,
                                             token = token)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:02<00:00,  1.41s/it]


<br>
<br>
We use lora method to fine-tune our model.
<br>
<br>

In [11]:
peft_params = LoraConfig(
    lora_alpha=64,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1000,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    #optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=1e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant"
)

In [12]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/Users/leo/.pyenv/versions/3.10.13/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/Users/leo/.pyenv/versions/3.10.13/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:222: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


Map: 100%|███████████████████████| 12000/12000 [00:01<00:00, 8490.87 examples/s]


<br>
<br>
Now the fine-tuned model is saved locally.
<br>
<br>

In [13]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

('Llama-2-7b-chat-hf-text2sql/tokenizer_config.json',
 'Llama-2-7b-chat-hf-text2sql/special_tokens_map.json',
 'Llama-2-7b-chat-hf-text2sql/tokenizer.json')